<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="../images/DLI_Header.png"></a></div>

**This is the Solution File for this notebook. Please try the [Challenge Version](../1-06_als_real_data.ipynb) first.**

# Alternating Least Squares with Real Data

Since we already have practice working with sparse data, we can apply the same techniques to our Amazon dataset. The trick is that because the data is so large, it will be trickier to use our debugging techniques from the <a href="1-05_als.ipynb"></a>previous notebook.

## Objectives
This notebook demonstrates:
* How to build a collaborative filter for large datasets
    * [1. Splitting into Train and Test](#1.-Splitting-into-Train-and-Test)
    * [2. Alternating Least Squares](#2.-Alternating-Least-Squares)
    * [3. Wrap Up](#3.-Wrap-Up)

## 1. Splitting into Train and Test    
We already did data cleansing in an earlier notebook, so let's load it up along with our libraries.

In [1]:
import cupy as cp
import cupyx.scipy.sparse
import cudf
import numpy as np

ratings = cudf.read_csv("../data/ratings.csv")

ratings.head()

,reviewerID,asin,overall,valid
0,AEVNTIQFU2TQ6,B00004ZCB3,3.0,False
1,A1XI6NT41B6E6X,B00004ZCB3,5.0,False
2,A2PMIMM3U3THM7,B00004ZCB3,5.0,False
3,A1E9NFMLMVY61H,B00004ZCB4,5.0,False
4,AXABTEYS7A4A8,B00004ZCB4,5.0,False


Before we split into train and test, let's [factorize](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.series.Series.factorize) our users and items. Our test set won't be effective if it doesn't use the same indexes as the training dataset.

In [2]:
user_indexes, user_mapping = ratings["reviewerID"].factorize()
item_indexes, item_mapping = ratings["asin"].factorize()

While we're building our sparse matrix, let's get a sense of our data along the way. First, let's see how many users there are.

In [3]:
user_count = user_mapping.count()

user_count

192403

Almost two hundred thousand! Okay, how about the items?

In [4]:
item_count = item_mapping.count()

item_count

63001

This is much bigger than our toy problem with four users and five items. Using [todense](https://docs-cupy.chainer.org/en/stable/reference/generated/cupyx.scipy.sparse.coo_matrix.html#cupyx.scipy.sparse.coo_matrix.todense) will result in memory limitations, which in the real world is both frustrating and expensive.

Instead, we'll need to rely on good testing. Let's break our ratings data down into train and test. Please fill in the `FIXME`s below, using the <a href="1-05_als_intro.ipynb">previous lab</a> as a hint if needed. There is one `FIXME` for each input of the `get_dataset` function.

In [5]:
# Train data
train_indexes = ~ratings["valid"]
valid_indexes = ratings["valid"]
overall = ratings["overall"]

def get_dataset(data_selector, user_indexes, item_indexes, overall):
    row = cp.asarray(user_indexes[data_selector])
    column = cp.asarray(item_indexes[data_selector])
    data = cp.asarray(overall[data_selector])
    
    sparse_data = cupyx.scipy.sparse.coo_matrix((data, (row, column)))
    mask = cp.asarray([1 for _ in range(len(data))], dtype=np.float32)
    sparse_mask = cupyx.scipy.sparse.coo_matrix((mask, (row, column)))
    return row, column, data, sparse_data, sparse_mask

train_row, train_column, train_data, train_sparse, train_mask = get_dataset(
    train_indexes, user_indexes, item_indexes, overall)

valid_row, valid_column, valid_data, valid_sparse, valid_mask = get_dataset(
    valid_indexes, user_indexes, item_indexes, overall)

## 2. Alternating Least Squares

It's time to initialize our embeddings. Let's copy over our functions from the previous lab.

In [6]:
embeddings = 2
shape = (user_count, item_count)

def initalize_features(length, embeddings):
    return cp.random.rand(embeddings, length) * 2 - 1

user_features = initalize_features(shape[0], embeddings)
item_features = initalize_features(shape[1], embeddings)

In [7]:
def rmse(user_features, item_features, data, row, column):
    predictions = (user_features[:, row] * item_features[:, column]).sum(axis=0)
    mean_squared_error = ((predictions - data) ** 2).mean() ** 0.5
    
    return predictions, mean_squared_error

The ALS function below is incomplete. Please fill in the `FIXME`s to get it up and running. There is one `FIXME` for each input.

In [8]:
def als(values, mask, features, scale=0.01):
    numerator = values.dot(features.T)
    squared_features = (features ** 2).sum(axis=0)
    denominator = scale + mask.dot(squared_features)

    return (numerator / denominator[:, None]).T

How low can you go? Run the below cell multiple times to watch the RMSE fall. If the above `FIXME`s are correctly implemented, the RMSE can reach below `1.25`.

In [9]:
for _ in range(20):
    user_features = als(train_sparse, train_mask, item_features)
    item_features = als(train_sparse.T, train_mask.T, user_features)
    predictions, error = rmse(
        user_features, item_features, valid_data, valid_row, valid_column)

    print ("RMSE:", error)

RMSE: 4.750533888189017
RMSE: 4.817339361099724
RMSE: 4.724361608649527
RMSE: 3.8984390421321558
RMSE: 2.1682555799806162
RMSE: 1.5670989597845253
RMSE: 1.3811294679964137
RMSE: 1.3111891395134707
RMSE: 1.2767704390718608
RMSE: 1.2435773412612998
RMSE: 1.223426514773521
RMSE: 1.2200548272731337
RMSE: 1.2195751822842507
RMSE: 1.214767974358177
RMSE: 1.2088431122969083
RMSE: 1.205333822457222
RMSE: 1.2041495747696696
RMSE: 1.2039103129322577
RMSE: 1.2038763506012367
RMSE: 1.2038716780724825


Let's do a human eyeball test to verify our prediction error. Do the predictions follow the same ranking order as the true ratings do?

In [10]:
valid_data[0:10]

array([4., 5., 5., 5., 5., 5., 1., 5., 2., 5.])

In [11]:
predictions[0:10]

array([4.37967407, 3.63454168, 5.19205639, 3.10017566, 4.73560372,
       3.39309085, 4.17715775, 4.43286414, 3.22058951, 3.98031958])

## 3. Wrap Up

Congratulations on finishing this set of notebooks! We learned how to do two major types of Recommender Systems: Content-based and Collaborative. While we were able to achieve a lower RMSE with Collaborative Filtering, there are some situations were Content-based wins out. Have an idea what they are? Head back over to the notebook launcher to take a quiz.

In [12]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="../images/DLI_Header.png"></a></div>